# Atividade


In [ ]:
distancias = [
    [0, 5, 9, 14, 7, 6, 12, 11, 8, 10, 13, 15],
    [5, 0, 4, 12, 6, 5, 11, 13, 9, 8, 14, 10],
    [9, 4, 0, 6, 10, 8, 12, 9, 7, 11, 13, 14],
    [14, 12, 6, 0, 8, 7, 9, 10, 12, 13, 5, 6],
    [7, 6, 10, 8, 0, 5, 8, 11, 10, 9, 12, 13],
    [6, 5, 8, 7, 5, 0, 6, 9, 8, 10, 11, 14],
    [12, 11, 12, 9, 8, 6, 0, 4, 7, 8, 10, 9],
    [11, 13, 9, 10, 11, 9, 4, 0, 3, 6, 7, 8],
    [8, 9, 7, 12, 10, 8, 7, 3, 0, 5, 9, 10],
    [10, 8, 11, 13, 9, 10, 8, 6, 5, 0, 4, 7],
    [13, 14, 13, 5, 12, 11, 10, 7, 9, 4, 0, 3],
    [15, 10, 14, 6, 13, 14, 9, 8, 10, 7, 3, 0]
]


In [ ]:
import random
import math

In [ ]:
maxCapacity = 8
volumes = [2, 1, 3, 2, 1, 3, 1, 2, 3, 1, 2, 1] #volumes de cada bairro fornecidos no pdf
numBairros = len(volumes)
minViagens = math.ceil(sum(volumes) / maxCapacity) # Adicionei uma conversão para inteiro usando o math
numPopulacao = 10 #Numero de cromossomos que serão criados de início
geracoes = 100 #Numero de gerações que serão criadas
taxaMutacao = 0.1

def cria_cromossomo():
  bairros = list(range(numBairros))
  random.shuffle(bairros)
  numViagens = random.randint(minViagens, numBairros - minViagens) #numero de viagens feitas gerado aleatoriamente
  for i in range(numViagens):
    posicao = random.randint(0, len(bairros))
    bairros.insert(posicao,"|") #A string inserida será um marcador usado para separar que bairros foram visitados em cada viagem
  return bairros

def cria_populacao(numPopulacao):
  populacao = []
  for i in range(numPopulacao):
    populacao.append(cria_cromossomo())
  return populacao

def fitness(cromossomo):
    viagens = []
    viagem = []
    for item in cromossomo:
        if item == "|":
            if viagem:
                viagens.append(viagem)
            viagem = []
        else:
            viagem.append(item)
    if viagem:
        viagens.append(viagem)

    total_distancia = 0
    penalidade = 0

    for viagem in viagens:
        carga = sum(volumes[b] for b in viagem)
        if carga > maxCapacity:
            penalidade += 1000  # penalidade por sobrecarga

        if viagem:
            distancia = distancias[0][viagem[0]]  # depósito para o primeiro bairro
            for i in range(len(viagem) - 1):
                distancia += distancias[viagem[i]][viagem[i+1]]
            distancia += distancias[viagem[-1]][0]  # volta ao depósito
            total_distancia += distancia

    penalidade += (len(viagens) - minViagens) * 100  # penalidade por excesso de viagens
    return total_distancia + penalidade

def crossover(pai1, pai2):
  #Remove o marcador
  p1 = [x for x in pai1 if x != '|']
  p2 = [x for x in pai2 if x != '|']

  #definindo pontos de corte
  a,b =sorted(random.sample(range(len(p1)), 2))

  filho_parcial = p1[a:b+1]
  restante = [x for x in p2 if x not in filho_parcial]
  filho = restante[:a] + filho_parcial + restante[a:]

  #inserindo marcadores no filho
  numViagens = random.randint(minViagens, numBairros - minViagens)
  for i in range(numViagens):
    posicao = random.randint(0, len(filho))
    filho.insert(posicao,"|")
  #Se vc achar melhor, pode fazer com que o marcador seja adicionado cada vez q a soma de x elementos baterem o max do caminhão. Até pq assim ele ta BEM aleatório
  return filho

def mutacao(cromossomo, taxa=0.1):
  # Mutação para garantir diversidade "genética"
  bairros = []
  for x in cromossomo:
    if x != "|":
      bairros.append(x)
  if random.random() < taxa:
    i, j = random.sample(range(len(bairros)), 2)
    bairros[i], bairros[j] = bairros[j], bairros[i]

  novoCromossomo = []
  carga = 0
  for b in bairros:
    if carga + volumes[b] > maxCapacity:
      novoCromossomo.append("|")
      carga = 0
    novoCromossomo.append(b)
    carga += volumes[b]
  return novoCromossomo

def algoritmo_genetico(numPopulacao, geracoes, taxaMutacao):
  populacao = cria_populacao(numPopulacao)
  for gen in range(geracoes):
      populacao.sort(key=fitness)
      nova_pop = populacao[:2]

      while len(nova_pop) < numPopulacao:
          pai1, pai2 = random.choices(populacao[:10], k=2)
          filho = crossover(pai1, pai2)
          filho = mutacao(filho, taxaMutacao)
          nova_pop.append(filho)

      populacao = nova_pop
      melhor_fitness = fitness(populacao[0])
      print(f"Geração {gen+1}: Melhor fitness = {melhor_fitness}")

  return populacao[0]

def print_viagens(cromossomo):
  viagens = []
  viagem = []
  for item in cromossomo:
      if item == "|":
          if viagem:
              viagens.append(viagem)
          viagem = []
      else:
          viagem.append(item)
  if viagem:
      viagens.append(viagem)

  print("\nRota Final:\n")
  for i, viagem in enumerate(viagens):
      bairros_string = ' -> '.join(f"B{b}" for b in viagem)
      carga = sum(volumes[b] for b in viagem)
      print(f"Viagem {i+1}: {bairros_string} - Carga: {carga}")
  print(f"Distância Total: {fitness(cromossomo):.2f}")

melhor = algoritmo_genetico(numPopulacao, geracoes, taxaMutacao)
print_viagens(melhor)




Geração 1: Melhor fitness = 375
Geração 2: Melhor fitness = 129
Geração 3: Melhor fitness = 117
Geração 4: Melhor fitness = 117
Geração 5: Melhor fitness = 111
Geração 6: Melhor fitness = 111
Geração 7: Melhor fitness = 111
Geração 8: Melhor fitness = 111
Geração 9: Melhor fitness = 111
Geração 10: Melhor fitness = 111
Geração 11: Melhor fitness = 106
Geração 12: Melhor fitness = 106
Geração 13: Melhor fitness = 106
Geração 14: Melhor fitness = 106
Geração 15: Melhor fitness = 106
Geração 16: Melhor fitness = 106
Geração 17: Melhor fitness = 106
Geração 18: Melhor fitness = 106
Geração 19: Melhor fitness = 104
Geração 20: Melhor fitness = 104
Geração 21: Melhor fitness = 104
Geração 22: Melhor fitness = 104
Geração 23: Melhor fitness = 104
Geração 24: Melhor fitness = 104
Geração 25: Melhor fitness = 98
Geração 26: Melhor fitness = 98
Geração 27: Melhor fitness = 98
Geração 28: Melhor fitness = 98
Geração 29: Melhor fitness = 98
Geração 30: Melhor fitness = 98
Geração 31: Melhor fitnes